In [ ]:
import pandas as pd
import ast

In [ ]:
# Read in the raw data
df = pd.read_csv('./data/bgg_stats_all.csv', low_memory=False)
len(df)

In [ ]:
# Remove these links as we're not doing anything with them
del df['thumbnail']
del df['image']

In [ ]:
# Function to parse cells containing strings that look like: "{'@value': '1986'}"
def parse_value(cell):
    try:
        new_val = ast.literal_eval(cell)
        new_val = new_val['@value']
        new_val = int(new_val)
    except:
        new_val = pd.NA
    return new_val

# Function to parse the name -- can be a dict or list of dicts
def parse_name(cell):
    name = ast.literal_eval(cell)
    if type(name) is list:
        name = name[0]
    return name['@value']

In [ ]:
# Get the year (published) of the board games by selecting the first available
# from 'yearpublished', 'datepublished', and 'releasedate', in that order.
df['isna_yearpublished'] = df['yearpublished'].isna()
df['isna_datepublished'] = df['datepublished'].isna()
df['isna_releasedate'] = df['releasedate'].isna()
df['year_yearpublished'] = df.apply(
    lambda row: pd.NA if row['isna_yearpublished'] else ast.literal_eval(row['yearpublished'])['@value'][0:4],
    axis=1
)
df['year_datepublished'] = df.apply(
    lambda row: pd.NA if row['isna_datepublished'] else ast.literal_eval(row['datepublished'])['@value'][0:4],
    axis=1
)
df['year_releasedate'] = df.apply(
    lambda row: pd.NA if row['isna_releasedate'] else ast.literal_eval(row['releasedate'])['@value'][0:4],
    axis=1
)
df['year'] = (
    df['year_yearpublished']
        .combine_first(df['year_datepublished'])
        .combine_first(df['year_releasedate'])
)

In [ ]:
# Clean up the year-related columns we no longer need
del df['isna_yearpublished']
del df['isna_datepublished']
del df['isna_releasedate']
del df['year_datepublished']
del df['year_releasedate']
del df['year_yearpublished']
del df['yearpublished']
del df['releasedate']
del df['datepublished']

df.info()

In [ ]:
df2 = df.copy()
# We need year and link, which contains mechanics/category info (among others)
# Drop any records that don't have either
df2.dropna(subset=['year', 'link'], inplace=True)

# Parse the name
df2['name'] = df2['name'].apply(parse_name)

# Convert year to int
df2 = df2.astype({'year': int})
len(df2)

In [ ]:
# Filter down to just board games

# All the @types
# ['boardgame', 'boardgameexpansion', 'boardgameaccessory',
#        'videogame', 'rpgitem', 'rpgissue', 'videogamecompilation',
#        'videogameexpansion', 'boardgameissue']

df2 = df2.loc[df2['@type'] == 'boardgame'].copy()
df2.info()

In [ ]:
# Parse the columns that are encoded as @key/@value dicts
cols = ['minplayers', 'maxplayers', 'playingtime', 'minplaytime', 'maxplaytime', 'minage']
for col in cols:
    df2[col] = df2[col].apply(parse_value)
df2.head()

In [ ]:
# Extract the value(s) of the link item with the provided key into a list.
# If the key isn't present, return pd.NA
def parse_link(link, key):
    link_list = ast.literal_eval(link)
    if isinstance(link_list, dict):
        if link_list['@type'] == key:
            return [link_list['@value']]
        return pd.NA
    if isinstance(link_list, list):
        filtered = list(
            filter(lambda x: x['@type'] == key,
            link_list
        ))
        return list(map(lambda x: x['@value'], filtered))
    return pd.NA

# Use parse_link to extract the category, mechanic, family, implementation,
# designer, artist, and publisher.
df2['categories'] = df2['link'].apply(lambda x: parse_link(x, 'boardgamecategory'))
df2['mechanics'] = df2['link'].apply(lambda x: parse_link(x, 'boardgamemechanic'))
df2['family'] = df2['link'].apply(lambda x: parse_link(x, 'boardgamefamily'))
df2['implementation'] = df2['link'].apply(lambda x: parse_link(x, 'boardgameimplementation'))
df2['designer'] = df2['link'].apply(lambda x: parse_link(x, 'boardgamedesigner'))
df2['artist'] = df2['link'].apply(lambda x: parse_link(x, 'boardgameartist'))
df2['publisher'] = df2['link'].apply(lambda x: parse_link(x, 'boardgamepublisher'))
# df2 = df2.astype({'year': int})

df2.head()

In [ ]:
# Export the results to cleaned.csv
df2.to_csv('data/cleaned2.csv', index=False)